<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [2]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [5]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [7]:
#ANSWER
response.status_code

200

In [9]:
#ANSWER
print(response.text)

{"message": "success", "timestamp": 1751636862, "iss_position": {"longitude": "-77.2499", "latitude": "-14.2504"}}


We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [12]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [15]:
#ANSWER
print(response.headers)

{'Date': 'Fri, 04 Jul 2025 13:48:28 GMT', 'Server': 'Apache', 'X-Amzn-Trace-Id': 'Root=1-6867dbac-4246168ea2c7747e954344bc;', 'Vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}


Print the content of the response (the data that the server returned):

In [18]:
#ANSWER
print(response.content)

b'{"api_status":"ALPHA","request_timestamp":"2025-07-04T13:48:28.960335399Z","norad_id":25544,"satellite_name":"ISS","tle_last_retrieved":"2025-07-03T21:20:07.811146437Z","lat":-33.87,"lon":151.21,"hours":24,"min_elevation":0,"query_ms":19,"passes":[{"start":"2025-07-04T23:05:08.941Z","tca":"2025-07-04T23:10:08.941Z","end":"2025-07-04T23:15:43.941Z","aos_azimuth":333,"los_azimuth":124,"max_elevation":33.0},{"start":"2025-07-05T00:42:03.941Z","tca":"2025-07-05T00:47:03.941Z","end":"2025-07-05T00:52:18.941Z","aos_azimuth":282,"los_azimuth":144,"max_elevation":21.0},{"start":"2025-07-05T02:21:28.941Z","tca":"2025-07-05T02:24:58.941Z","end":"2025-07-05T02:28:33.941Z","aos_azimuth":234,"los_azimuth":153,"max_elevation":5.0},{"start":"2025-07-05T04:00:33.941Z","tca":"2025-07-05T04:03:33.941Z","end":"2025-07-05T04:06:28.941Z","aos_azimuth":205,"los_azimuth":139,"max_elevation":3.0},{"start":"2025-07-05T05:37:03.941Z","tca":"2025-07-05T05:41:33.941Z","end":"2025-07-05T05:46:13.941Z","aos_azimu

Note that this is a Python byte string:

In [21]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [24]:
#ANSWER
print(response.headers['content-type'])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [27]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2025-07-04T13:48:28.960335399Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2025-07-03T21:20:07.811146437Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 19, 'passes': [{'start': '2025-07-04T23:05:08.941Z', 'tca': '2025-07-04T23:10:08.941Z', 'end': '2025-07-04T23:15:43.941Z', 'aos_azimuth': 333, 'los_azimuth': 124, 'max_elevation': 33.0}, {'start': '2025-07-05T00:42:03.941Z', 'tca': '2025-07-05T00:47:03.941Z', 'end': '2025-07-05T00:52:18.941Z', 'aos_azimuth': 282, 'los_azimuth': 144, 'max_elevation': 21.0}, {'start': '2025-07-05T02:21:28.941Z', 'tca': '2025-07-05T02:24:58.941Z', 'end': '2025-07-05T02:28:33.941Z', 'aos_azimuth': 234, 'los_azimuth': 153, 'max_elevation': 5.0}, {'start': '2025-07-05T04:00:33.941Z', 'tca': '2025-07-05T04:03:33.941Z', 'end': '2025-07-05T04:06:28.941Z', 'aos_azimuth': 205, 'los_azimuth': 139, 'max_elevation': 3.0}, {'start': '2025-07-05T05:37:03.941Z', 'tca': 

What kind of object did this give us? Dictionary

In [30]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate list:

In [33]:
#ANSWER:
passes = overheads['passes']

Now extract the `start` strings into an array called `srisetimes`:

In [36]:
#ANSWER:
srisetimes = [xpass['start'] for xpass in passes]

These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [39]:
srisetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]

Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [41]:
#ANSWER:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]
risetimes

[datetime.datetime(2025, 7, 4, 23, 5, 8, 941000),
 datetime.datetime(2025, 7, 5, 0, 42, 3, 941000),
 datetime.datetime(2025, 7, 5, 2, 21, 28, 941000),
 datetime.datetime(2025, 7, 5, 4, 0, 33, 941000),
 datetime.datetime(2025, 7, 5, 5, 37, 3, 941000),
 datetime.datetime(2025, 7, 5, 7, 13, 23, 941000),
 datetime.datetime(2025, 7, 5, 8, 51, 13, 941000),
 datetime.datetime(2025, 7, 5, 22, 17, 3, 941000)]

Finally, here is an endpoint that tells us who is on board:

In [44]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [47]:
#ANSWER:
astros = response.json()
print(astros)
print(astros["number"])
for astronaut in astros['people']:
    print(astronaut['name'])

{'people': [{'craft': 'ISS', 'name': 'Oleg Kononenko'}, {'craft': 'ISS', 'name': 'Nikolai Chub'}, {'craft': 'ISS', 'name': 'Tracy Caldwell Dyson'}, {'craft': 'ISS', 'name': 'Matthew Dominick'}, {'craft': 'ISS', 'name': 'Michael Barratt'}, {'craft': 'ISS', 'name': 'Jeanette Epps'}, {'craft': 'ISS', 'name': 'Alexander Grebenkin'}, {'craft': 'ISS', 'name': 'Butch Wilmore'}, {'craft': 'ISS', 'name': 'Sunita Williams'}, {'craft': 'Tiangong', 'name': 'Li Guangsu'}, {'craft': 'Tiangong', 'name': 'Li Cong'}, {'craft': 'Tiangong', 'name': 'Ye Guangfu'}], 'number': 12, 'message': 'success'}
12
Oleg Kononenko
Nikolai Chub
Tracy Caldwell Dyson
Matthew Dominick
Michael Barratt
Jeanette Epps
Alexander Grebenkin
Butch Wilmore
Sunita Williams
Li Guangsu
Li Cong
Ye Guangfu


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [51]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code,
    '''
    if response == 200:
        return False, response
    else:
        return True, response
    
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [54]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')

response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)[0]

API call failed. Resolve issue before continuing!


True

>



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



